In [1]:
import numpy as np
import cv2
from skimage.feature import hog, local_binary_pattern
from os import listdir, mkdir
from os.path import isfile, join, dirname, exists
from PIL import Image
import os
# from sklearn.svm import SVC
from scipy.signal import convolve2d
# from sklearn.externals import joblib
# import joblib
import matplotlib.pyplot as pyplot


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'fonts-dataset\\Scheherazade New'

In [67]:
IBM_plex_path = "Processed-fonts-dataset/fonts-dataset/IBM Plex Sans Arabic/"
Lemonada_path = "Processed-fonts-dataset/fonts-dataset/Lemonada/"
Marhey_path = "Processed-fonts-dataset/fonts-dataset/Marhey/"
Scheherazade_path = "Processed-fonts-dataset/fonts-dataset/Scheherazade New/"
debug = False


In [68]:
IBM_plex_features = []
Lemonada_features = []
Marhey_features = []
Scheherazade_features = []

## Preprocessing Module

In [69]:

#rotate the image with given theta value
def rotate(img, theta):
    rows, cols = img.shape[0], img.shape[1]
    image_center = (cols/2, rows/2)
    
    M = cv2.getRotationMatrix2D(image_center,theta,1)

    abs_cos = abs(M[0,0])
    abs_sin = abs(M[0,1])

    bound_w = int(rows * abs_sin + cols * abs_cos)
    bound_h = int(rows * abs_cos + cols * abs_sin)

    M[0, 2] += bound_w/2 - image_center[0]
    M[1, 2] += bound_h/2 - image_center[1]

    print(img[0][0])
    # rotate orignal image to show transformation
    print(img[0][0])
    rotated = cv2.warpAffine(img,M,(bound_w,bound_h),borderValue=(int(img[0][0]),int(img[0][0]),int(img[0][0])))
    return rotated
def round_to_angle_multiples(number):

    # Define the possible angles
    angles = [45, -45, 90, -90, 135, -135]

    # Calculate the absolute difference between the number and each angle
    differences = [abs(number - angle) for angle in angles]

    # Find the index of the angle with the smallest difference
    min_index = differences.index(min(differences))

    # Return the corresponding angle
    return angles[min_index]


def rotate_img(edgeimage,originalimage):
    # Apply Hough Lines Transform
    lines = cv2.HoughLinesP(edgeimage, rho=1, theta=np.pi/180, threshold=100, minLineLength=50, maxLineGap=10)
    
    finalImage=originalimage
    # Find the longest line
    longest_line = None
    max_length = 0
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # Calculate line length using distance formula
            length = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            if length > max_length:
                max_length = length
                longest_line = line

        if longest_line is not None:
            x1, y1, x2, y2 = longest_line[0]
            print(x1,y1,x2,y2)
            if x2 - x1 != 0:  # Avoid division by zero
                 orientation = np.arctan((y2 - y1) / (x2 - x1))*180  # Radians
            else:
                orientation = 180/2  # Vertical line (slope is undefined)
            # print("oo",orientation)
            if (orientation>20.0 or orientation<-20):
                orientation=round_to_angle_multiples(orientation)
                finalImage = rotate(originalimage, 180-orientation)
        
        # # print(orientation)
        # # Draw only the longest line (if any)
        # if longest_line is not None:
        #     x1, y1, x2, y2 = longest_line[0]
        #     cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 2)  # Draw blue line with thickness 2
        #     display(img,"box")

    return finalImage

In [70]:
def preprocess(image_path):
    # Read the image
    img = cv2.imread(image_path)

    # Salt and pepper noise
    img=cv2.medianBlur(img,5)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # display(img)

    # cv2.imshow('Detected Lines (Longest)', img)
    # Apply Canny edge detection
    edges = cv2.Canny(img, 20, 150)  # Adjust threshold values as needed

    rotated_img=rotate_img(edges,img)
    # display(img)
    if (img[0][0]>200):
        _, img = cv2.threshold(rotated_img, 50.0, 255.0, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
        print("white")
    else:
        _, img = cv2.threshold(rotated_img, 50.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        print("black")
    # display(img)
    return img

def process_image(image_path, output_dir):
    # print("fonts-dataset//IBM Plex Sans Arabic//0.jpeg")
    img = preprocess(image_path)  # Assuming preprocess returns a NumPy array
    img = Image.fromarray(img)  # Convert to PIL Image object
    new_path = join(output_dir, image_path.split("/")[-1])
    img.save(new_path)
    
def process_folder(folder_path):
  # output_dir = join(dirname(folder_path), "processed")
  # if not exists(output_dir):
  #   mkdir(output_dir)  # Create output folder if it doesn't exist
  # for f in listdir(folder_path):
  #   print(f)
  # print
  for folder in listdir(folder_path):
    img = join(folder_path, folder)
    # print(img)
    if isfile(img) and img.lower().endswith(".jpeg"):
      # Check for image files with common extensions
      process_image(img, "Processed-fonts-dataset")    

In [71]:
# process_folder("fonts-dataset\IBM Plex Sans Arabic")
# process_folder("fonts-dataset\Lemonada")
# process_folder("fonts-dataset\Marhey")
# process_folder("fonts-dataset\Scheherazade New")

## Feature Extraction

In [72]:
def SIFT_features(img,limit:bool = True,max_keypoints:int = 1000):

    sift = cv2.SIFT_create()
    keypoints_sift, descriptors = sift.detectAndCompute(img, None)
    if limit:
        if len(keypoints_sift) > max_keypoints:
            keypoints_sift = keypoints_sift[:max_keypoints]
            descriptors = descriptors[:max_keypoints]

    # Convert keypoints and descriptors to numpy arrays
    keypoints_array = np.float32([kp.pt for kp in keypoints_sift])
    descriptors_array = np.float32(descriptors)

    return keypoints_sift, descriptors

In [73]:
def lpq(img_filepath, winSize=3,freqestim=1,mode='nh'):

	img = cv2.imread(img_filepath,0)
	rho=0.90

	STFTalpha=1/winSize  # alpha in STFT approaches (for Gaussian derivative alpha=1)
	sigmaS=(winSize-1)/4 # Sigma for STFT Gaussian window (applied if freqestim==2)
	sigmaA=8/(winSize-1) # Sigma for Gaussian derivative quadrature filters (applied if freqestim==3)

	convmode='valid' # Compute descriptor responses only on part that have full neigborhood. Use 'same' if all pixels are included (extrapolates np.image with zeros).

	img=np.float64(img) # Convert np.image to double
	r=(winSize-1)/2 # Get radius from window size
	x=np.arange(-r,r+1)[np.newaxis] # Form spatial coordinates in window

	if freqestim==1:  #  STFT uniform window
	    #  Basic STFT filters
			w0=np.ones_like(x)
			w1=np.exp(-2*np.pi*x*STFTalpha*1j)
			w2=np.conj(w1)

	## Run filters to compute the frequency response in the four points. Store np.real and np.imaginary parts separately
	# Run first filters
	filterResp1=convolve2d(convolve2d(img,w0.T,convmode),w1,convmode)
	filterResp2=convolve2d(convolve2d(img,w1.T,convmode),w0,convmode)
	filterResp3=convolve2d(convolve2d(img,w1.T,convmode),w1,convmode)
	filterResp4=convolve2d(convolve2d(img,w1.T,convmode),w2,convmode)

	# Initilize frequency domain matrix for four frequency coordinates (np.real and np.imaginary parts for each frequency).
	freqResp=np.dstack([filterResp1.real, filterResp1.imag,
	                    filterResp2.real, filterResp2.imag,
	                    filterResp3.real, filterResp3.imag,
	                    filterResp4.real, filterResp4.imag])

	## Perform quantization and compute LPQ codewords
	inds = np.arange(freqResp.shape[2])[np.newaxis,np.newaxis,:]
	LPQdesc=((freqResp>0)*(2**inds)).sum(2)

	## Histogram if needed
	if mode=='nh' or mode=='h':
	    LPQdesc=np.histogram(LPQdesc.flatten(),range(256))[0]

	## Normalize histogram if needed
	if mode=='nh':
	    LPQdesc=LPQdesc/LPQdesc.sum()

	# print (LPQdesc)
	LPQdesc = np.array(LPQdesc)
	return LPQdesc

In [74]:
def is_image_number(filename, target_number):
  #Checks if the filename corresponds to a specific image number.
  prefix = str(target_number) + "_"
  return filename.startswith(prefix)

In [75]:
# # IBM_plex_features_after_modification = []
for filename in os.listdir(IBM_plex_path):
    if filename.endswith(".jpeg"):
        image_path = os.path.join(IBM_plex_path, filename)
        imageLPQ = lpq(image_path)  # Assuming lpq is defined elsewhere
        IBM_plex_features.append(imageLPQ)
        # print(imageLPQ)
        print(f"Processed image: {filename}")
        print("-----------------------------------")
        

IBM_plex_features = np.array(IBM_plex_features) 
IBM_plex_labels = np.full(len(IBM_plex_features),3)  
np.save('IBM_plex_features_h.npy', IBM_plex_features)
np.save('IBM_plex_labels_h.npy', IBM_plex_labels)

Processed image: 0.jpeg
-----------------------------------
Processed image: 1.jpeg
-----------------------------------
Processed image: 10.jpeg
-----------------------------------
Processed image: 100.jpeg
-----------------------------------
Processed image: 101.jpeg
-----------------------------------
Processed image: 102.jpeg
-----------------------------------
Processed image: 103.jpeg
-----------------------------------
Processed image: 104.jpeg
-----------------------------------
Processed image: 105.jpeg
-----------------------------------
Processed image: 106.jpeg
-----------------------------------
Processed image: 107.jpeg
-----------------------------------
Processed image: 108.jpeg
-----------------------------------
Processed image: 109.jpeg
-----------------------------------
Processed image: 11.jpeg
-----------------------------------
Processed image: 110.jpeg
-----------------------------------
Processed image: 111.jpeg
-----------------------------------
Processed imag

In [76]:
# # IBM_plex_features_after_modification = []
for filename in os.listdir(Lemonada_path):
    if filename.endswith(".jpeg"):
        image_path = os.path.join(Lemonada_path, filename)
        imageLPQ = lpq(image_path)  # Assuming lpq is defined elsewhere
        Lemonada_features.append(imageLPQ)
        # print(imageLPQ)
        print(f"Processed image: {filename}")
        print("-----------------------------------")
        

Lemonada_features = np.array(Lemonada_features) 
Lemonada_labels = np.full(len(Lemonada_features),2)  
np.save('Lemonada_features_h.npy', Lemonada_features)
np.save('Lemonada_labels_h.npy', Lemonada_labels)


Processed image: 0.jpeg
-----------------------------------
Processed image: 1.jpeg
-----------------------------------
Processed image: 10.jpeg
-----------------------------------
Processed image: 100.jpeg
-----------------------------------
Processed image: 101.jpeg
-----------------------------------
Processed image: 102.jpeg
-----------------------------------
Processed image: 103.jpeg
-----------------------------------
Processed image: 104.jpeg
-----------------------------------
Processed image: 105.jpeg
-----------------------------------
Processed image: 106.jpeg
-----------------------------------
Processed image: 107.jpeg
-----------------------------------
Processed image: 108.jpeg
-----------------------------------
Processed image: 109.jpeg
-----------------------------------
Processed image: 11.jpeg
-----------------------------------
Processed image: 110.jpeg
-----------------------------------
Processed image: 111.jpeg
-----------------------------------
Processed imag

In [77]:
for filename in os.listdir(Scheherazade_path):
    if filename.endswith(".jpeg"):
        image_path = os.path.join(Scheherazade_path, filename)
        imageLPQ = lpq(image_path)  # Assuming lpq is defined elsewhere
        Scheherazade_features.append(imageLPQ)
        print(f"Processed image: {filename}")
        print("-----------------------------------")

Scheherazade_features = np.array(Scheherazade_features) 
Scheherazade_labels = np.full(len(Scheherazade_features),0)  
np.save('Scheherazade_features_h.npy', Scheherazade_features)
np.save('Scheherazade_labels_h.npy', Scheherazade_labels)

Processed image: 0.jpeg
-----------------------------------
Processed image: 1.jpeg
-----------------------------------
Processed image: 10.jpeg
-----------------------------------
Processed image: 100.jpeg
-----------------------------------
Processed image: 101.jpeg
-----------------------------------
Processed image: 102.jpeg
-----------------------------------
Processed image: 103.jpeg
-----------------------------------
Processed image: 104.jpeg
-----------------------------------
Processed image: 105.jpeg
-----------------------------------
Processed image: 106.jpeg
-----------------------------------
Processed image: 107.jpeg
-----------------------------------
Processed image: 108.jpeg
-----------------------------------
Processed image: 109.jpeg
-----------------------------------
Processed image: 11.jpeg
-----------------------------------
Processed image: 110.jpeg
-----------------------------------
Processed image: 111.jpeg
-----------------------------------
Processed imag

In [ ]:
for filename in os.listdir(Marhey_path):
    if filename.endswith(".jpeg"):
        image_path = os.path.join(Marhey_path, filename)
        imageLPQ = lpq(image_path)  # Assuming lpq is defined elsewhere
        Marhey_features.append(imageLPQ)
        print(f"Processed image: {filename}")
        print("-----------------------------------")

Marhey_features = np.array(Marhey_features) 
Marhey_labels = np.full(len(Marhey_features),1)  
np.save('Marhey_features_h.npy', Marhey_features)
np.save('Marhey_labels_h.npy', Marhey_labels)

Processed image: 0.jpeg
-----------------------------------
Processed image: 1.jpeg
-----------------------------------
Processed image: 10.jpeg
-----------------------------------
Processed image: 100.jpeg
-----------------------------------
Processed image: 101.jpeg
-----------------------------------
Processed image: 102.jpeg
-----------------------------------
Processed image: 103.jpeg
-----------------------------------
Processed image: 104.jpeg
-----------------------------------
Processed image: 105.jpeg
-----------------------------------
Processed image: 106.jpeg
-----------------------------------
Processed image: 107.jpeg
-----------------------------------
Processed image: 108.jpeg
-----------------------------------
Processed image: 109.jpeg
-----------------------------------
Processed image: 11.jpeg
-----------------------------------
Processed image: 110.jpeg
-----------------------------------
Processed image: 111.jpeg
-----------------------------------
Processed imag